<a href="https://colab.research.google.com/github/MarisaRies/covir/blob/main/COVIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
#Encryption
#!pip install cape_privacy

In [97]:
################
#Datenbankaufbau
################

#Lade Bibliotheken
import sqlite3 as sq
from sqlite3 import Error
import pandas as pd
import numpy as np
import requests
import datetime as dt

#Pfad Definitionen
drive_path = "/content/drive"
material_path = f"{drive_path}/MyDrive/Studium/Material"
csv_file_path = f"{material_path}/csv_data/covid19/"

#Verbinden von GDrive
from google.colab import drive 
drive.mount("/content/drive", force_remount=True)

#Verbindung zu SQLite
def create_connection_memory():
  conn = None;
  try:
    #Verbindungsaufbau
    conn = sq.connect(':memory:')
    return conn
    print(sqlite3.version)
  except Error as e:
    print("Fehler #1 beim Verbinden zu SQLite", e)

#Lokale Definition der SQLite Verbindung
def create_connection_local(local_path):
  conn = None;
  try:
    #Verbindungsaufbau
    conn = sq.connect(local_path+'/Studium/covir.db')
    return conn
    print(sqlite3.version)
  except Error as e:
    print("Fehler #2 beim Verbinden zu SQLite", e) 
conn = create_connection_memory()

cur = conn.cursor()
print("Verbindung erfolgreich zu SQLite")

#Tabellenschema erzeugen
sql_create_source_data = requests.get('https://raw.githubusercontent.com/MarisaRies/covir/main/sql/db_create_statements.sql').text
cur.executescript(sql_create_source_data)
conn.commit()
print("Tabellenschema erfolgreich erstellt")



Mounted at /content/drive
Verbindung erfolgreich zu SQLite
Tabellenschema erfolgreich erstellt


In [98]:
# ETL-Prozess
#
# Extraktion
#

#CSV-Daten aus Google-Drive laden, relevante Spalten auswählen und in Datenbank laden
df_insert_pat = pd.read_csv(csv_file_path+'patients.csv')
df_insert_pat = df_insert_pat[['Id','BIRTHDATE','DEATHDATE','FIRST','LAST']]

df_insert_enc = pd.read_csv(csv_file_path+'encounters.csv')
df_insert_enc = df_insert_enc[['Id','START','STOP','PATIENT','CODE','DESCRIPTION']]

df_insert_con = pd.read_csv(csv_file_path+'conditions.csv')
df_insert_con = df_insert_con[['ENCOUNTER','START','STOP','PATIENT','CODE','DESCRIPTION']]

df_insert_imm = pd.read_csv(csv_file_path+'immunizations.csv')
df_insert_imm = df_insert_imm[['ENCOUNTER','DATE','PATIENT','CODE','DESCRIPTION']]

#
# Transformation
#

df_insert_pat['BIRTHDATE'] = pd.to_datetime(df_insert_pat['BIRTHDATE'], format='%Y-%m-%d')
df_insert_pat['DEATHDATE'] = pd.to_datetime(df_insert_pat['DEATHDATE'], format='%Y-%m-%d')

df_insert_pat['EOL'] = df_insert_pat.DEATHDATE
df_insert_pat['EOL'] = df_insert_pat['EOL'].fillna(pd.to_datetime("today"))

df_insert_pat['AGE'] =  df_insert_pat['EOL'].dt.year - df_insert_pat['BIRTHDATE'].dt.year
df_insert_pat = df_insert_pat.drop(columns=['EOL'])

#Erstelle zentrale View für das Stern-Schema
cur.executescript(requests.get('https://raw.githubusercontent.com/MarisaRies/covir/main/sql/ETLview_create_statement.sql').text)
conn.commit()

#
# Laden
#

df_insert_pat.to_sql('tb_patients', conn, if_exists='append', index=False)
df_insert_enc.to_sql('tb_encounters', conn, if_exists='append', index=False)
df_insert_con.to_sql('tb_conditions', conn, if_exists='append', index=False)
df_insert_imm.to_sql('tb_immunizations', conn, if_exists='append', index=False)

conn.commit()

print("CSV-Daten erfolgreich geladen, Datenbank bereit")

CSV-Daten erfolgreich geladen, Datenbank bereit


In [103]:
df_insert_pat.describe()
df_insert_enc.describe()
df_insert_con.describe()
df_insert_imm.describe()

#df_insert_pat.info()
#df_insert_enc.info()
#df_insert_con.info()
#df_insert_imm.info()

,CODE
count,13125.000000
mean,110.235581
std,45.916130
min,3.000000
25%,113.000000
50%,140.000000
75%,140.000000
max,140.000000
